In [1]:
import os
import sys
import pandas as pd
from importlib import reload

In [2]:
import enviserv.dictan
from enviserv.dictan import DictAnalyzer
import pandserv as pds # сервисные функции для пандас и не только

In [3]:
import Connection as cn

In [4]:
spark = cn.spark_app_builder(cn.get_spark_master_ip(),
                             spark_app_name = "pyspark-taxi-forecasting")

INFO:ch:spark_master_ip: 172.18.0.2
DEBUG:ch:starting import SparkSession
INFO:ch:starting building spark app object: pyspark-taxi-forecasting
INFO:ch:builded spark app object: <pyspark.sql.session.SparkSession object at 0x7fd13c1f9210>


In [5]:
%%time
taxi = spark.read.load('/work/data/Taxi_Trips_-_2022.csv', 
                       format='csv', header='true', inferSchema='true')

CPU times: user 17.4 ms, sys: 1.84 ms, total: 19.3 ms
Wall time: 42 s


In [6]:
%%time
# Вывести схему данных
taxi.printSchema()

root
 |-- Trip ID: string (nullable = true)
 |-- Taxi ID: string (nullable = true)
 |-- Trip Start Timestamp: string (nullable = true)
 |-- Trip End Timestamp: string (nullable = true)
 |-- Trip Seconds: integer (nullable = true)
 |-- Trip Miles: double (nullable = true)
 |-- Pickup Census Tract: long (nullable = true)
 |-- Dropoff Census Tract: long (nullable = true)
 |-- Pickup Community Area: integer (nullable = true)
 |-- Dropoff Community Area: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Tips: double (nullable = true)
 |-- Tolls: double (nullable = true)
 |-- Extras: double (nullable = true)
 |-- Trip Total: double (nullable = true)
 |-- Payment Type: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Pickup Centroid Latitude: double (nullable = true)
 |-- Pickup Centroid Longitude: double (nullable = true)
 |-- Pickup Centroid Location: string (nullable = true)
 |-- Dropoff Centroid Latitude: double (nullable = true)
 |-- Dropoff Centroid 

In [7]:
pd.set_option('display.max_colwidth', None)  # Показать полное содержимое ячеек столбцов

In [8]:
table_descr = pd.read_csv('/work/data/table_descr.csv',sep=';',index_col='Column Name')

table_descr

,Description,Type,Описание,Тип
Column Name,,,,
Trip ID,A unique identifier for the trip.,Plain Text,Уникальный идентификатор поездки,Обычный текст
Taxi ID,A unique identifier for the taxi.,Plain Text,Уникальный идентификатор такси,Обычный текст
Trip Start Timestamp,When the trip started rounded to the nearest 15 minutes.,Date & Time,Время начала поездки округленное до ближайших 15 минут,Дата и время
Trip End Timestamp,When the trip ended rounded to the nearest 15 minutes.,Date & Time,Время окончания поездки округленное до ближайших 15 минут,Дата и время
Trip Seconds,Time of the trip in seconds.,Number,Продолжительность поездки в секундах,Число
Trip Miles,Distance of the trip in miles.,Number,Расстояние поездки в милях,Число
Pickup Census Tract,The Census Tract where the trip began. For privacy this Census Tract is not shown for some trips. This column often will be blank for locations outside Chicago.,Plain Text,Код района по переписи населения где началась поездка. В целях конфиденциальности этот код не отображается для некоторых поездок. Это поле часто остается пустым для мест за пределами Чикаго.,Обычный текст
Dropoff Census Tract,The Census Tract where the trip ended. For privacy this Census Tract is not shown for some trips. This column often will be blank for locations outside Chicago.,Plain Text,Код района по переписи населения где завершилась поездка. В целях конфиденциальности этот код не отображается для некоторых поездок. Это поле часто остается пустым для мест за пределами Чикаго.,Обычный текст
Pickup Community Area,The Community Area where the trip began. This column will be blank for locations outside Chicago.,Number,Номер коммунальной области где началась поездка. Это поле остается пустым для мест за пределами Чикаго.,Число


In [9]:
pd.reset_option('display.max_colwidth')  # Сброс максимальной ширины столбцов к значению по умолчанию

In [14]:
# %%time
# describe_result = taxi.describe().toPandas()

CPU times: user 408 ms, sys: 122 ms, total: 530 ms
Wall time: 4min 33s


In [15]:
# output_file_path = '/work/data/2022_describe.csv'
# describe_result.to_csv(output_file_path, index=False) #transpose().

In [10]:
describe_result_22 = pd.read_csv('/work/data/2022_describe.csv',sep=',',index_col='summary')

In [11]:
describe_result_22_formatted = describe_result_22.copy()
pds.get_df_formated(describe_result_22_formatted, '`',2, 10)

In [12]:
describe_result_22_formatted.transpose()

summary,count,mean,stddev,min,max
Trip ID,6`382`425,nan,nan,000000bb18,ffffff1aae
Taxi ID,6`382`425,nan,nan,0041f8f0c9,fff84aa08a
Trip Start Timestamp,6`382`425,nan,nan,01/01/2022,12/31/2022
Trip End Timestamp,6`382`213,nan,nan,01/01/2022,12/31/2022
Trip Seconds,6`380`960,1`198.21,1`895.66,0,86`341
Trip Miles,6`382`369,6.19,8.00,0,2`967.54
Pickup Census Tract,2`623`831,17`031`468`160.38,368`945.90,17`031`010`100,17`031`980`100
Dropoff Census Tract,2`675`331,17`031`411`846.86,345`773.49,17`031`010`100,17`031`980`100
Pickup Community Area,5`868`572,32.35,25.20,1,77
Dropoff Community Area,5`748`741,25.84,20.93,1,77


In [13]:
val = int(describe_result_22.transpose()['count'].astype(float).max())

In [14]:
print(pds.gvf(val))

6'382'425


In [19]:
%%time
row_count = taxi.count()

CPU times: user 3.32 ms, sys: 2.62 ms, total: 5.94 ms
Wall time: 11.9 s


In [20]:
print(pds.gvf(row_count))

6'382'425


In [17]:
%%time
taxi.freqItems(['Pickup Census Tract']).show()

+-----------------------------+
|Pickup Census Tract_freqItems|
+-----------------------------+
|         [17031842900, 170...|
+-----------------------------+

CPU times: user 10.9 ms, sys: 0 ns, total: 10.9 ms
Wall time: 16.4 s


In [21]:
# Функция для вычисления частот
def frq(df, column):
    # Вычисляем частоту для каждого значения
    freq_df = df.groupBy(column).count().orderBy("count", ascending=False)
    return freq_df

In [20]:
%%time
pct_frq = frq(taxi,'Pickup Census Tract')
pct_frq.show()

+-------------------+-------+
|Pickup Census Tract|  count|
+-------------------+-------+
|               null|3758594|
|        17031980000| 519802|
|        17031320100| 304178|
|        17031839100| 284131|
|        17031281900| 188178|
|        17031081500| 174424|
|        17031081403| 112656|
|        17031330100| 108691|
|        17031081401| 104198|
|        17031980100|  85226|
|        17031081201|  84359|
|        17031081300|  76482|
|        17031320400|  71039|
|        17031081700|  63163|
|        17031320600|  61120|
|        17031280100|  55753|
|        17031841000|  41201|
|        17031081800|  39526|
|        17031081402|  35200|
|        17031081600|  30245|
+-------------------+-------+
only showing top 20 rows

CPU times: user 10.1 ms, sys: 10.4 ms, total: 20.5 ms
Wall time: 16.5 s


In [25]:
%%time
dct_frq = frq(taxi,'Dropoff Census Tract')
dct_frq.show()

+--------------------+-------+
|Dropoff Census Tract|  count|
+--------------------+-------+
|                null|3707094|
|         17031839100| 328855|
|         17031320100| 277879|
|         17031980000| 243652|
|         17031081500| 160251|
|         17031330100| 139399|
|         17031081403| 133973|
|         17031281900| 129117|
|         17031081700| 114125|
|         17031081401| 100596|
|         17031081201|  87949|
|         17031833000|  83912|
|         17031320400|  81112|
|         17031081800|  76118|
|         17031081300|  75221|
|         17031280100|  67209|
|         17031320600|  59928|
|         17031980100|  48559|
|         17031841000|  42231|
|         17031081600|  40664|
+--------------------+-------+
only showing top 20 rows

CPU times: user 16.4 ms, sys: 11.6 ms, total: 28.1 ms
Wall time: 14.2 s


In [26]:
%%time
pca_frq = frq(taxi,'Pickup Community Area')
pca_frq.show()

+---------------------+-------+
|Pickup Community Area|  count|
+---------------------+-------+
|                    8|1482215|
|                   32|1031010|
|                   76| 989907|
|                   28| 531280|
|                 null| 513853|
|                   33| 229646|
|                    6| 229347|
|                   56| 183449|
|                    7| 150637|
|                    3| 100679|
|                   77|  76036|
|                   24|  75849|
|                    2|  47266|
|                   41|  44448|
|                   35|  35625|
|                    1|  35251|
|                   43|  33720|
|                   38|  33032|
|                   22|  31390|
|                   16|  30553|
+---------------------+-------+
only showing top 20 rows

CPU times: user 17.6 ms, sys: 860 µs, total: 18.5 ms
Wall time: 15.2 s


In [15]:
def compare_unique_values(df, field1, field2):
    # Извлекаем уникальные значения для каждого поля
    unique_values_field1 = df.select(field1).distinct().collect()
    unique_values_field2 = df.select(field2).distinct().collect()
    
    # Преобразуем результаты в множества
    set_field1 = {row[field1] for row in unique_values_field1}
    set_field2 = {row[field2] for row in unique_values_field2}
    
    # Находим значения, которые есть в set_field1, но нет в set_field2
    only_in_field1 = {value for value in set_field1 if value not in set_field2}
    
    # Находим значения, которые есть в set_field2, но нет в set_field1
    only_in_field2 = {value for value in set_field2 if value not in set_field1}
    
    return only_in_field1, only_in_field2, set_field1, set_field2

In [16]:
%%time
only_pickup, only_dropoff, pickup, dropoff = \
compare_unique_values(taxi, 'Pickup Census Tract','Dropoff Census Tract')

CPU times: user 81.6 ms, sys: 1.13 ms, total: 82.7 ms
Wall time: 33 s


In [39]:
print('`Pickup Census Tract` distinct count:',len(pickup), ' | only in pickup set:',len(only_pickup))
print('`Dropoff Census Tract` distinct count:',len(dropoff), ' | only in dropoff set:',len(only_dropoff))

`Pickup Census Tract` distinct count: 666  | only in pickup set: 34
`Dropoff Census Tract` distinct count: 812  | only in dropoff set: 180


In [27]:
unique_count = taxi.select('Pickup Census Tract').distinct().count()
print(f"Количество уникальных элементов в столбце 'Pickup Census Tract': {unique_count}")

Количество уникальных элементов в столбце 'Pickup Census Tract': 666


In [28]:
unique_count = taxi.select('Dropoff Census Tract').distinct().count()
print(f"Количество уникальных элементов в столбце 'Dropoff Census Tract': {unique_count}")

Количество уникальных элементов в столбце 'Dropoff Census Tract': 812


In [17]:
%%time
taxi.take(1)[0]

CPU times: user 5.36 ms, sys: 11 ms, total: 16.3 ms
Wall time: 205 ms


Row(Trip ID='bcfa19f2539021c054809d4c3993d226996ae095', Taxi ID='368ce5511598af2cc07efdb68067d381174fdac1d47a2837d021e647f4c8febb3d9bf330e155845b8217b9b451c29b1b9ecf5d55d4bfe6cfd5d79ffb49b1f334', Trip Start Timestamp='01/01/2022 12:00:00 AM', Trip End Timestamp='01/01/2022 12:00:00 AM', Trip Seconds=152, Trip Miles=0.1, Pickup Census Tract=None, Dropoff Census Tract=None, Pickup Community Area=None, Dropoff Community Area=None, Fare=3.75, Tips=0.0, Tolls=0.0, Extras=0.0, Trip Total=3.75, Payment Type='Cash', Company='Medallion Leasin', Pickup Centroid Latitude=None, Pickup Centroid Longitude=None, Pickup Centroid Location=None, Dropoff Centroid Latitude=None, Dropoff Centroid Longitude=None, Dropoff Centroid  Location=None)

In [18]:
%%time
pck = taxi.select('Pickup Census Tract').distinct().collect()

CPU times: user 15.5 ms, sys: 1.55 ms, total: 17.1 ms
Wall time: 14.1 s


In [19]:
%%time
taxi.createOrReplaceTempView("taxis")
result = spark.sql("SELECT DISTINCT `Pickup Census Tract` FROM taxis")
pck2 = result.collect()

CPU times: user 15.1 ms, sys: 0 ns, total: 15.1 ms
Wall time: 14.1 s


In [20]:
import time
from tqdm import tqdm

In [21]:
# Количество испытаний
num_trials = 5

# Время выполнения для первого блока кода: сбор уникальных значений с помощью DataFrame API
pck_times = []
for _ in tqdm(range(num_trials)):
    start_time = time.time()
    pck = taxi.select('Pickup Census Tract').distinct().collect()
    end_time = time.time()
    pck_times.append(end_time - start_time)

# Вывод среднего времени выполнения для первого блока кода
print("Среднее время выполнения для первого блока кода: {:.2f} секунд".format(sum(pck_times) / num_trials))

100%|██████████| 5/5 [01:07<00:00, 13.52s/it]

Среднее время выполнения для первого блока кода: 13.52 секунд


In [22]:
# Время выполнения для второго блока кода: сбор уникальных значений с помощью SQL запроса
result_times = []
for _ in tqdm(range(num_trials)):
    start_time = time.time()
    taxi.createOrReplaceTempView("taxis")
    result = spark.sql("SELECT DISTINCT `Pickup Census Tract` FROM taxis")
    pck2 = result.collect()
    end_time = time.time()
    result_times.append(end_time - start_time)

# Вывод среднего времени выполнения для второго блока кода
print("Среднее время выполнения для второго блока кода: {:.2f} секунд".format(sum(result_times) / num_trials))


100%|██████████| 5/5 [01:05<00:00, 13.04s/it]

Среднее время выполнения для второго блока кода: 13.03 секунд


In [40]:
f1 = 'Pickup Census Tract'
f3 = 'Pickup Centroid Latitude'
f2 = 'Pickup Centroid Longitude'
pckgeo = taxi.select(f1, f2, f3).distinct().collect()

In [42]:
len(pckgeo)

743

In [44]:
from collections import defaultdict

# Инициализация словаря для хранения значений
tract_to_coords = defaultdict(list)

# Заполнение словаря
for row in pckgeo:
    tract = row[f1]
    coords = (row[f3], row[f2])
    tract_to_coords[tract].append(coords)

# Поиск районов с разными значениями широты и долготы
repeated_tracts = {tract: coords for tract, coords in tract_to_coords.items() if len(set(coords)) > 1}

# Вывод результатов
for tract, coords in repeated_tracts.items():
    print(f"Район: {tract}, Координаты: {coords}")
    print()

# Количество таких районов
print(f"Количество районов с разными значениями широты и долготы: {len(repeated_tracts)}")


Район: None, Координаты: [(41.694878966, -87.713192497), (41.779582888, -87.768510849), (41.80908443, -87.632424524), (41.874005383, -87.66351755), (41.9867118, -87.663416405), (41.80901825, -87.659166599), (41.761577908, -87.572781987), (41.953582125, -87.72345239), (41.660136051, -87.60284764), (41.944226601, -87.655998182), (42.001571027, -87.695012589), (42.007612593, -87.813781034), (41.771848515, -87.695666342), (41.835117986, -87.618677767), (41.707311449, -87.534902901), (41.899602111, -87.633308037), (41.890608853, -87.756046711), (None, None), (41.929297368, -87.798032181), (41.690633347, -87.570058269), (41.706125752, -87.598255838), (41.745757713, -87.708365704), (41.706587882, -87.623366512), (41.968069, -87.721559063), (41.94651142, -87.806020002), (41.792981903, -87.724208194), (41.985015101, -87.804532006), (41.744205146, -87.656305986), (41.740205756, -87.615969523), (41.947791586, -87.683834942), (41.713148612, -87.675075312), (41.993930128, -87.758353588), (41.729676

In [45]:
f3 = 'Pickup Centroid Latitude'
f2 = 'Pickup Centroid Longitude'
geos_pck = taxi.select(f2, f3).distinct().collect()

In [46]:
len(geos_pck)

616

In [47]:
geos_pck

[Row(Pickup Centroid Longitude=-87.701941074, Pickup Centroid Latitude=41.97681175),
 Row(Pickup Centroid Longitude=-87.713192497, Pickup Centroid Latitude=41.694878966),
 Row(Pickup Centroid Longitude=-87.768510849, Pickup Centroid Latitude=41.779582888),
 Row(Pickup Centroid Longitude=-87.632424524, Pickup Centroid Latitude=41.80908443),
 Row(Pickup Centroid Longitude=-87.630865027, Pickup Centroid Latitude=41.905857769),
 Row(Pickup Centroid Longitude=-87.760884371, Pickup Centroid Latitude=41.912836784),
 Row(Pickup Centroid Longitude=-87.710715958, Pickup Centroid Latitude=41.69507035),
 Row(Pickup Centroid Longitude=-87.66351755, Pickup Centroid Latitude=41.874005383),
 Row(Pickup Centroid Longitude=-87.571184739, Pickup Centroid Latitude=41.755301636),
 Row(Pickup Centroid Longitude=-87.6200147, Pickup Centroid Latitude=41.76196108),
 Row(Pickup Centroid Longitude=-87.632746489, Pickup Centroid Latitude=41.880994471),
 Row(Pickup Centroid Longitude=-87.565823879, Pickup Centroid